In [171]:
import numpy as np

In [172]:
testinput=[
"Button A: X+94, Y+34",
"Button B: X+22, Y+67",
"Prize: X=8400, Y=5400",
"",
"Button A: X+26, Y+66",
"Button B: X+67, Y+21",
"Prize: X=12748, Y=12176",
"",
"Button A: X+17, Y+86",
"Button B: X+84, Y+37",
"Prize: X=7870, Y=6450",
"",
"Button A: X+69, Y+23",
"Button B: X+27, Y+71",
"Prize: X=18641, Y=10279",
]

In [173]:
def convertinput(ipt):
    out = []
    A = None
    B = None
    C = None
    for l in ipt:
        if "A" in l:
            A = [ int(l[l.find("X")+2:l.find(",")]), int(l[l.find("Y")+2:]) ]
        if "B" in l:
            B = [ int(l[l.find("X")+2:l.find(",")]), int(l[l.find("Y")+2:]) ]
        if "Prize" in l:
            C = [ int(l[l.find("X")+2:l.find(",")]), int(l[l.find("Y")+2:]) ]
            out.append([A, B, C])
    return out

In [174]:
def playGame(i, costA, costB):
    A, B, Win = i 
    # When too far
    maxA = int(max(Win[0]/A[0], Win[1]/A[1]))+1
    maxB = int(max(Win[0]/B[0], Win[1]/B[1]))+1
    if costA > costB:
        #try to go as many with B first
        for i in range(maxB+1):
            goB = maxB-i
            if B[0]*goB>Win[0] or B[1]*goB>Win[1]: continue
            for goA in range(maxA+1):
                X = B[0]*goB+A[0]*goA
                Y = B[1]*goB+A[1]*goA
                if X>Win[0] or Y>Win[1]: continue
                if X == Win[0]:
                    if Y == Win[1]: return True, goA, goB, goA*costA+goB*costB
                    #else: return False, -1, -1, -1
    else:
        #try to go as many with B first
        for i in range(maxA+1):
            goA = maxA-i
            if A[0]*goA>Win[0] or A[1]*goA>Win[1]: continue
            for goB in range(maxB+1):
                X = B[0]*goB+A[0]*goA
                Y = B[1]*goB+A[1]*goA
                if X>Win[0] or Y>Win[1]: continue
                if X == Win[0]:
                    if Y == Win[1]: return True, goA, goB, goA*costA+goB*costB
                    #else: return False, -1, -1, -1
    return False, -1, -1, -1

In [175]:
tipt = convertinput(testinput)

In [176]:
ipt = convertinput([l.strip("\n") for l in open("input.txt")])

In [177]:
cost = 0
for i in tipt:
    out = playGame(i, 3, 1)
    print(out)
    if out[0]: cost+=out[3]
print(cost)

(True, 80, 40, 280)
(False, -1, -1, -1)
(True, 38, 86, 200)
(False, -1, -1, -1)
480


In [178]:
cost = 0
for i in ipt:
    out = playGame(i, 3, 1)
    #print(out)
    if out[0]: cost+=out[3]
print(cost)

29522


In [183]:
def solveNumpy(i, costA=3, costB=1):
    A, B, W = i
    a = np.array([[A[0], B[0]], [A[1], B[1]]])
    b = np.array([W[0], W[1]])
    x = np.linalg.solve(a, b)
    
    NA = round(x[0], 4)
    NB = round(x[1], 4)
    if NA % 1 == 0 and NB % 1 == 0:
        NA=int(NA)
        NB=int(NB)
        return NA, NB, NA*costA + NB*costB
    return x[0], x[1], -1
    
def playGamesNumpy(ipt, corr = 0, prt=True):
    cost = 0
    for i in ipt:
        A, B, W = i
        i2 = [A, B, [W[0]+corr, W[1]+corr] ]
        na, nb, c = solveNumpy(i2)
        if c>0: cost+=c
        if prt: print(na,nb, c)
    print(cost)

In [184]:
playGamesNumpy(tipt)

80 40 280
141.40454076367388 135.3952528379773 -1
38 86 200
244.5016362786349 65.56989247311829 -1
480


In [185]:
playGamesNumpy(tipt, 10000000000000)

81081081161.08107 108108108148.10812 -1
118679050709 103199174542 459236326669
71266110727.91661 104624715779.70735 -1
102851800151 107526881786 416082282239
875318608908


In [204]:
playGamesNumpy(ipt, 10000000000000, prt=False)

101214869433312


In [188]:
A, B, W = tipt[0]
a = np.array([[A[0], B[0]], [A[1], B[1]]])
b = np.array([W[0], W[1]])
x = np.linalg.solve(a, b)
x

array([80., 40.])

In [208]:
def NAfromNB(i, NB):
    A, B, W = i
    return W[0]/A[0] - NB*(B[0]/A[0])

def NBfromW(i):
    A, B, W = i
    return (W[1] - W[0]*A[1]/A[0]) / ( B[1]-B[0]*A[1]/A[0] )

def solveSmart(i, costA=3, costB=1):
    A, B, W = i
    #rounding ugly but needed here even more, to much precision loss...
    NB = round(NBfromW(i), 2)
    NA = round(NAfromNB(i, NB), 2)
    if NA % 1 == 0 and NB % 1 == 0:
        NA=int(NA)
        NB=int(NB)
        return NA, NB, NA*costA + NB*costB
    return NA, NB, -1
    
def playGamesSmart(ipt, corr = 0, prt=True):
    cost = 0
    for i in ipt:
        A, B, W = i
        i2 = [A, B, [W[0]+corr, W[1]+corr] ]
        na, nb, c = solveSmart(i2)
        if c>0: cost+=c
        if prt: print(na,nb, c)
    print(cost)

In [209]:
playGamesSmart(ipt, 10000000000000, prt=False)

101214869433312
